In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

In [3]:
seed=1
np.random.seed(seed)
tf.random.set_seed(seed)

In [5]:
from sklearn.preprocessing import QuantileTransformer

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [13]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train.target
test_id = test.id
X_test = test.drop(['id'], axis=1)

In [14]:
qt = QuantileTransformer()
X_train = pd.DataFrame(qt.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(qt.transform(X_test), columns=X_train.columns)

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=20,
    verbose=0,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=.2,
    patience=5,
    mode='min'
)

In [17]:
from tensorflow.keras import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Dense, Flatten, InputLayer, Dropout
from tensorflow.keras.optimizers import Adam

In [66]:
def get_model():
    inp = Input(shape=X_train.shape[1], name='input')
    h = Dense(128, activation='swish')(inp)
    h = Dropout(.25)(h)
    h = Dense(64, activation='swish')(h)
    h = Dropout(.25)(h)
    h = Dense(32, activation='swish')(h)
    h = Dropout(.25)(h)
    h = Dense(1, activation='swish')(h)
    
    model = Model(inputs=inp, outputs=h)
    model.compile(optimizer=Adam(learning_rate=1e-3),
                 loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-3),
                 metrics=['AUC'])
    return model

In [19]:
from sklearn.model_selection import StratifiedKFold

In [70]:
from sklearn.model_selection import StratifiedKFold

EPOCHS = 100
BATCH_SIZE = 1024
VERBOSE = 1
N_SPLITS = 5

### cross-validation 
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=seed)

scores = {fold:None for fold in range(cv.n_splits)}
predictions = []

for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train)):
    Xtrain, ytrain = X_train.iloc[idx_train], y_train[idx_train]
    Xvalid, yvalid = X_train.iloc[idx_valid], y_train[idx_valid]

    model = get_model()

    print('**'*20)
    print(f"Fold {fold+1} || Training")
    print('**'*20)
    
    history = model.fit(
        Xtrain, ytrain,
        validation_data=(Xvalid, yvalid),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=VERBOSE,
        callbacks=[
            early_stopping,
            reduce_lr
        ]
    )
    
    scores[fold] = (history.history)
    
    print(f"Fold {fold+1} || Max Validation AUC: {np.max(scores[fold]['val_auc'])}")
    
    prediction = model.predict(X_test, batch_size=BATCH_SIZE).reshape(1,-1)[0]
    predictions.append(prediction)

print('**'*20)
print('Finished Training')
print('**'*20)

overall_auc = [np.max(scores[fold]['val_auc']) for fold in range(cv.n_splits)]
print('Overall Mean AUC: ', np.mean(overall_auc))

****************************************
Fold 1 || Training
****************************************
Epoch 1/100


InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (model_14/dense_58/IdentityN:0) = ] [[0.135584235][-0.0792661831][-0.0350260586]...] [y (Cast_3/x:0) = ] [0]
	 [[node assert_greater_equal/Assert/AssertGuard/Assert
 (defined at C:\Users\user\anaconda3\lib\site-packages\keras\utils\metrics_utils.py:608)
]] [Op:__inference_train_function_136031]

Errors may have originated from an input operation.
Input Source operations connected to node assert_greater_equal/Assert/AssertGuard/Assert:
In[0] assert_greater_equal/Assert/AssertGuard/Assert/assert_greater_equal/All:	
In[1] assert_greater_equal/Assert/AssertGuard/Assert/data_0:	
In[2] assert_greater_equal/Assert/AssertGuard/Assert/data_1:	
In[3] assert_greater_equal/Assert/AssertGuard/Assert/data_2:	
In[4] assert_greater_equal/Assert/AssertGuard/Assert/model_14/dense_58/IdentityN:	
In[5] assert_greater_equal/Assert/AssertGuard/Assert/data_4:	
In[6] assert_greater_equal/Assert